### Setting Up the Database and Stage 

In [ ]:
-- Create Stage for Analyst

USE  DATABASE CHATBOT_WORKSHOP_DB;


USE CHATBOT_WORKSHOP_DB;

CREATE OR REPLACE STAGE DOCS ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );


-- Create External Stage 
CREATE OR REPLACE STAGE CORTEX_SEARCH_EXT_STG
url = 's3://holworkshopbucket/Cortex-Search/'
file_format = (type = csv);

### Copy the Source PDF File into Docs Stage from the Cortex Search External Stage

In [ ]:
COPY FILES INTO @CORTEX_ANALYST_STG FROM @CORTEX_ANALYST_EXT_STG/ticket.yaml;

### Creating a python function to chunk the pdf document into vectors

In [ ]:
create or replace function text_chunker(pdf_text string)
returns table (chunk varchar)
language python
runtime_version = '3.9'
handler = 'text_chunker'
packages = ('snowflake-snowpark-python', 'langchain')
as
$$
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

class text_chunker:

    def process(self, pdf_text: str):
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 1512, # Adjust this as you see fit
            chunk_overlap  = 256, # This enables text have some form of overlap. Useful for keeping chunks contextual
            length_function = len
        )
    
        chunks = text_splitter.split_text(pdf_text)
        df = pd.DataFrame(chunks, columns=['chunks'])
        
        yield from df.itertuples(index=False, name=None)
$$;


### Listing the file in the docs stage

In [ ]:
ls @docs;

### Create the table where we are going to store the chunks extracted from the PDF.



In [ ]:
create or replace TABLE DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the PDF file
    SIZE NUMBER(38,0), -- Size of the PDF
    FILE_URL VARCHAR(16777216), -- URL for the PDF
    SCOPED_FILE_URL VARCHAR(16777216), -- Scoped url (you can choose which one to keep depending on your use case)
    CHUNK VARCHAR(16777216), -- Piece of text
    CATEGORY VARCHAR(16777216) -- Will hold the document category to enable filtering
);

### Function SNOWFLAKE.CORTEX.PARSE_DOCUMENT will be used to read the PDF documents directly from the staging area. The text will be passed to the function previously created to split the text into chunks. There is no need to create embeddings as that will be managed automatically by Cortex Search service later.


In [ ]:
insert into docs_chunks_table (relative_path, size, file_url,
                            scoped_file_url, chunk)

    select relative_path, 
            size,
            file_url, 
            build_scoped_file_url(@docs, relative_path) as scoped_file_url,
            func.chunk as chunk
    from 
        directory(@docs),
        TABLE(text_chunker (TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(@docs, relative_path, {'mode': 'LAYOUT'})))) as func;

In [ ]:
select * from docs_chunks_table;

### Creating a temp table to load the category of each document using the COMPLETE function

In [ ]:
CREATE
OR REPLACE TEMPORARY TABLE docs_categories AS WITH unique_documents AS (
  SELECT
    DISTINCT relative_path
  FROM
    docs_chunks_table
),
docs_category_cte AS (
  SELECT
    relative_path,
    TRIM(snowflake.cortex.COMPLETE (
      'llama3-70b',
      'Given the name of the file between <file> and </file> determine if it is related to banking services or customer support or tech support.
       Very strictly, Use only one word <file> ' || relative_path || '</file>'
    ), '\n') AS category
  FROM
    unique_documents
)
SELECT
  *
FROM
  docs_category_cte;

In [ ]:
SELECT
  *
FROM
  docs_category_cte;

### Update the Chunks Table with the  identified category

In [ ]:
update docs_chunks_table 
  SET category = docs_categories.category
  from docs_categories
  where  docs_chunks_table.relative_path = docs_categories.relative_path;

### Verify the tables

In [ ]:
select * from docs_categories;
select * from docs_chunks_table;

### Cortex Search is created and enabled on the chunk column

In [ ]:
create or replace CORTEX SEARCH SERVICE CUSTOMER_SEARCH_SERVICE
ON chunk
ATTRIBUTES CATEGORY
warehouse = COMPUTE_WH
TARGET_LAG = '1 minute'
as (
    select chunk,
        relative_path,
        file_url,
        category
    from docs_chunks_table
);
